In [1]:
import csv
import pandas as pd
import psycopg2

In [2]:
#Приведение CSV файла к корректному формату
movieId = []
movieName = []
movieYear = []
with open('IMDBMovieTitles.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        try:
            mn, not_good_y = row[1].split('(')
            y, bad_y = not_good_y.split(')')
        except:
            continue
 
        movieId.append(row[0])
        movieName.append(mn.strip())
        movieYear.append(y)

In [3]:
#Сохранение правильного CSV файла
df = pd.DataFrame({'id':   movieId,
                   'year': movieYear, 
                   'name': movieName})
df.to_csv('newCSV.csv', index=False)

In [4]:
#Connection
conn = psycopg2.connect(database='lab1', user='postgres',
    password='12345', host='localhost')#, port=5432)

In [5]:
#Создание новой таблицы в Postgres
'''''''''
cur = conn.cursor()
cur.execute("create table movies (id integer generated by default as " + 
            "identity primary key, year integer, name text)")

conn.commit()
cur.close()
'''''''''

'\ncur = conn.cursor()\ncur.execute("create table movies (id integer generated by default as " + \n            "identity primary key, year integer, name text)")\n\nconn.commit()\ncur.close()\n'

In [6]:
#Функция поиска
def find(text, connection):
    txt = text.split()
    year = ''
    
    #Поиск года в запросе
    for word in txt:
        if word.isnumeric() and len(word) == 4:
            try:
                if int(word) >= 1894:
                    year = word
                    break
            except:
                continue
                
    #Если год есть, то удалем его из запроса
    if year != '':
        text = text.replace(year, '', 1)
    
    #Странное изменение строки для того, чтобы работали SQL запросы
    text = text.strip()
    text = text.replace(' ', '% %')
    text = '%' + text + '%'
    
    #Создание курсора
    cur = connection.cursor()
    
    #SQL запросы
    ret = []

    if text == '%%':
        ret.append('Введите поисковой запрос.')
        return ret
    elif year == '':
        cur.execute('Select * from public.movies where name like %(название фильма)s limit 10', 
                    {'название фильма': text})
    elif text == '':
        cur.execute('Select * from public.movies where year = %(дата выхода)s limit 10',
                   {'дата выхода': year})
    else:
        cur.execute('Select * from movies where name like %(название фильма)s and year = %(дата выхода)s limit 10',
                   {'название фильма': text, 'дата выхода': year})
        
    results = cur.fetchall()
    
    #Закрытие курсора
    cur.close()
    
    for res in results:
        ret.append('Название фильма: ' + res[2] +
                   ' Дата выхода: '    + str(res[1]) +
                   ' Id фильма: '      + str(res[0]))
    
    if len(ret) == 0:
        ret.append('Ничего не найдено.')
        
    return ret

In [8]:
searchQuery = str(input('Введите поисковой запрос: '))
results = find(searchQuery, conn)
for result in results:
    print(result)

Введите поисковой запрос: ass 1990
Название фильма: Class of 1999 Дата выхода: 1990 Id фильма: 99277
Название фильма: The Gate II Trespassers Дата выхода: 1990 Id фильма: 99636
Название фильма: Leatherface Texas Chainsaw Massacre III Дата выхода: 1990 Id фильма: 99994
